# Managing Schemas

The Bailo python client enables intuitive interaction with the Bailo service, from within a python environment. This example notebook will run through the following concepts:

* Creating a new schema on Bailo.
* Retrieving schemas from the service.

Prerequisites:

* Python 3.8.1 or higher (including a notebook environment for this demo).
* A local or remote Bailo service (see https://github.com/gchq/Bailo).

## Introduction

The Bailo python client is split into two sub-packages: **core** and **helper**.

* **Core:** For direct interactions with the service endpoints.
* **Helper:** For more intuitive interactions with the service, using classes (e.g. Model) to handle operations.

In order to create helper classes, you will first need to instantiate a `Client()` object from the core. By default, this object will not support any authentication. However, Bailo also supports PKI authentication, which you can use from Python by passing a `PkiAgent()` object into the `Client()` object when you instantiate it.

In [ ]:
# Necessary import statements

from bailo import Client, Schema, SchemaKind
import random
import string

# Instantiating the PkiAgent(), if using.
# agent = PkiAgent(cert='', key='', auth='')

# Instantiating the Bailo client

client = Client("http://127.0.0.1:8080") # <- INSERT BAILO URL (if not hosting locally)

## Creating a new schema

In this section, we'll create a new schema using the `Schema.create()` classmethod. On the Bailo service, a schema must consist of 5 parameters upon creation. These are **schema_id**, **name**, **description**, **kind** and **json_schema**. Below, we use the `Client()` object created before when instantiating the new `Schema()` object. In this instance, **schema_id** is generated using a random generator function, however any unique identifier would suffice.

NOTE: This creates the schema on your Bailo service too! This operation is typically reserved for administrators.

In [ ]:
def random_generator(N=10):
    return "".join(random.choices(string.ascii_uppercase + string.digits, k=N))

json_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "overview": {
            "title": "Overview",
            "description": "Summary of the model functionality.",
            "type": "object",
            "properties": {
                "modelOverview": {
                    "title": "What does the model do?",
                    "description": "A description of what the model does.",
                    "type": "string",
                    "minLength": 1,
                    "maxLength": 5000,
                },
                "tags": {
                    "title": "Descriptive tags for the model.",
                    "description": "These tags will be searchable and will help others find this model.",
                    "type": "array",
                    "widget": "tagSelectorBeta",
                    "items": {"type": "string"},
                    "uniqueItems": True,
                },
            },
            "required": [],
            "additionalProperties": False,
        },
    },
    "required": [],
    "additionalProperties": False,
}

schema = Schema.create(
    client=client,
    schema_id=random_generator(),
    name="Test",
    description="Example Description",
    kind=SchemaKind.MODEL,
    json_schema=json_schema
)

schema_id = schema.schema_id 

## Retrieving an existing schema

### Using the .from_id() method

In this section, we'll retrieve our previous schema using the `Schema.from_id()` classmethod. This will create your `Schema()` object as before, but using existing information retrieved from the service.

In [ ]:
schema = Schema.from_id(client=client, schema_id=schema_id)

schema.description